In [1]:
import os
import openeo
import calendar
import numpy as np
import xarray as xr
import pandas as pd

In [2]:
# Connection and authentication
con = openeo.connect("openeo.dataspace.copernicus.eu")
con.authenticate_oidc()

# Definition of the region 
spatial_extent = {"west": -6.2239, "south": 42.2967, "east": -6.1679, "north": 42.3367 ,"crs": "EPSG:4326",} # mini tabuyo
year = '2023'

os.makedirs(year, exist_ok=True)

temporal_extent = [year+"-01-01", year+"-12-31"]
datacube = con.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    max_cloud_cover=10,
    bands=["B04", "B03", "B02", "B08", "SCL"],
)

print(datacube)
#nc_path = f"{year}/sentinel2_{year}.nc"
nc_path = f"datacubes/tabuyo/{year}/mini/sentinel2_mini_{year}.nc"
datacube.download(nc_path)


Authenticated using refresh token.
DataCube(<PGNode 'load_collection' at 0x78869ba88b80>)


### Code for downloading the netCDF (.nc) files by month

In [8]:
def download_and_plot_monthly_data(year):

    for month in range(1, 13):  # January to December
        print('Month: ',month)
        month_str = str(month).zfill(2)  # Zero-padding the month to ensure two digits
        num_days = calendar.monthrange(year, month)[1]  # Get the number of days in the month
        start_date = f"{year}-{month_str}-01"
        end_date = f"{year}-{month_str}-{num_days}"
        
        month_datacube = datacube.filter_temporal(start_date, end_date)

        #nc_path = f"{year}/sentinel2_{year}-{month_str}.nc"
        nc_path = f"datacubes/tabuyo/{year}/mini/sentinel2_mini_{year}-{month_str}.nc"
        month_datacube.download(nc_path)

        '''
        # Show the first and last images of the month
        ds = xarray.load_dataset(nc_path)
        data = ds[["B04", "B03", "B02"]].to_array(dim="bands")
        
        fig, axes = plt.subplots(ncols=2, figsize=(8, 3), dpi=300, sharey=True)
        data[{"t": 0}].plot.imshow(vmin=0, vmax=2000, ax=axes[0])
        data[{"t": -1}].plot.imshow(vmin=0, vmax=2000, ax=axes[1])
        
        plt.show()
        '''
        if month==5:
            break
        
download_and_plot_monthly_data(int(year))


Month:  1
Month:  2
Month:  3
Month:  4
Month:  5


### Code for downloading the geoTiff (.tif) file for each day of the year that meets the requirements

In [10]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin, Affine

def save_to_tif(ds, output_dir):
    for time in ds.t:
        date = pd.to_datetime(time.values)
        date_str = date.strftime('%Y-%m-%d')
        filename = os.path.join(output_dir, f"{date_str}.tif")

        # Select the bands for red, green, and blue channels
        red = ds.sel(t=time)['B04']
        green = ds.sel(t=time)['B03']
        blue = ds.sel(t=time)['B02']

        # Stack the bands into an RGB image
        rgb_image = np.stack([red, green, blue], axis=-1)

        profile = {
            'driver': 'GTiff',
            'count': 3,
            'dtype': 'int16',
            'width': ds.dims['x'],
            'height': ds.dims['y'],
            'crs': 'EPSG:32629', #"EPSG:4326",
            #'transform': from_origin(float(ds.x.min()), float(ds.y.max()), float(ds.x[1] - ds.x[0]), float(ds.y[1] - ds.y[0])),
            #'transform': from_origin(726460.0, 4693660.0, 10, 10), # ESTO SOLO VALE PARA TABUYO
            'transform': from_origin(728705.0, 4691045.0, 10, 10), # ESTO SOLO VALE PARA MINI TABUYO
        }
        
        with rasterio.open(filename, 'w', **profile) as dst:
            dst.write(rgb_image.transpose(2, 0, 1))

output_dir = f"datacubes/tabuyo/{year}/mini"
os.makedirs(output_dir, exist_ok=True)
ds = xr.open_dataset(f"datacubes/tabuyo/{year}/mini/sentinel2_mini_{year}.nc")

save_to_tif(ds, output_dir)


/tmp/ipykernel_9961/3846078461.py:23: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'width': ds.dims['x'],
/tmp/ipykernel_9961/3846078461.py:24: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'height': ds.dims['y'],
/home/bea/.local/lib/python3.10/site-packages/numpy/core/_asarray.py:126: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
